In [1]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv


In [2]:
CONFIG = {'title':'S3E26_try_13'\
         , 'DCRP':'Stacking'}

## import

In [3]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.distributions import uniform
from scipy.special import softmax

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import SGDClassifier as SGD
from sklearn.linear_model import LogisticRegression as LR
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score


In [5]:
!pip install optuna

In [6]:
import optuna
import lightgbm as lgb
from lightgbm import LGBMClassifier as lgbmc

In [7]:
!pip install keras-tuner -q

In [8]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import keras_tuner

In [9]:
from imblearn.under_sampling import RandomUnderSampler

## def

In [10]:
def func_logloss(df_t0, df_p0):
    '''df_t0: Series/Ground_truth, df_p0: DataFrame/Prediction'''
#     if type(df_t0)==pd.Series:
#         df_t0 = df_t0.to_frame()
#     if type(df_p0)==pd.Series:
#         df_p0 = df_p0.to_frame()
    df_p1 = df_p0.applymap(lambda x:max(min(x,1-1e-15),1e-15))
    N = df_t0.shape[0]
    p_sum = 0
    for ans in df_t0.unique():
#         print(ans)
        idx_ok = df_t0[df_t0==ans].index
    #     print(idx_ok)
    #     print(df_p1.loc[idx_ok, ans].transform(np.log))
        p_temp = (df_p1.loc[idx_ok, ans].transform(np.log).sum())
    #     print(p_temp)
        p_sum += p_temp
    score = (-1)*p_sum/N
    return score

In [11]:
def func_result(df_t0, df_p0):
    df_t1 = df_t0.rename('y')
    df_p1 = df_p0.apply(lambda x: np.argmax(x),axis='columns').rename('p')
    df_s0 = pd.concat([df_t1,df_p1],axis='columns')
    df_s1 = df_s0.apply(lambda x:x[0]==x[1],axis='columns').value_counts()
    print(df_s1)
    print('{:.03}'.format(df_s1[0]/(df_s1[0]+df_s1[1])))   

## Data

In [12]:
train = pd.read_csv(os.path.join(dirname,'train.csv')).set_index('id')
test = pd.read_csv(os.path.join(dirname,'test.csv')).set_index('id')
lst_cols =  train.columns.to_list()
np.array(lst_cols)

array(['N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly',
       'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin',
       'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets',
       'Prothrombin', 'Stage', 'Status'], dtype='<U13')

## Preprocess phase 0

In [13]:
df_X_train = train.drop(['Status'],axis='columns').copy()
df_X_test = test.copy()
df_y = train['Status'].copy()

def func_prepro_0(df_a0,df_b0):
    def func_N_Years(df_0):
        df_1 = df_0['N_Days']
        df_2 = (df_1/365).transform(np.ceil).astype(int).rename('N_Years')
        df_3 = pd.concat([df_2,df_0],axis='columns')
        return df_3
    df_a1 = func_N_Years(df_a0)
    df_b1 = func_N_Years(df_b0)
    
    def func_Age_Years(df_0):
        df_1 = df_0['Age']
        df_2 = (df_1/365).transform(np.floor).astype(int).rename('Age_Years')
        df_3 = pd.concat([df_2,df_0],axis='columns')
        return df_3
    df_a2 = func_Age_Years(df_a1)
    df_b2 = func_Age_Years(df_b1)

    def func_float(df_0):
        df_1 = df_0.select_dtypes(include='float')
        df_2 = df_1.apply(lambda x: np.log(1+x)).add_prefix('log_')
        df_3 = pd.concat([df_0,df_2],axis='columns')
        return df_3
    df_a3 = func_float(df_a2)
    df_b3 = func_float(df_b2)
    
    def func_obj(df_0):
        df_1 = df_0.select_dtypes(include='object')
        df_2 = df_1.astype('category').add_prefix('cate_')
        df_3 = pd.DataFrame(index=df_0.index)
        for col in df_1.columns:
            df_4 = pd.get_dummies(df_1[col],dtype=int,prefix=f'dmy_{col}')
            df_3 = pd.concat([df_3,df_4],axis='columns')
        df_5 = pd.concat([df_2,df_3,df_0],axis='columns')
        return df_5
    df_a4 = func_obj(df_a3)
    df_b4 = func_obj(df_b3)
    
#     def poly_float(df_0):
#         df_1 = df_0.select_dtypes(include='float')
#         lst_col_0 = df_1.columns.to_list()
#         poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=True)
#         arr_2 = poly.fit_transform(df_1)
#         df_2 = pd.DataFrame(arr_2\
#                             , index=df_1.index\
#                             , columns=poly.get_feature_names_out())
#         df_3 = df_2.drop(lst_col_0+['1'],axis='columns')
#         col_0 = df_3.columns.to_list()
#         col_1 = [re.sub(' ','_',s) for s in col_0]
#         df_3.columns = col_1
#         df_3 = df_3.add_prefix('poly_')
#         df_4 = pd.concat([df_0,df_3],axis='columns')
#         return df_4
#     df_a5 = poly_float(df_a4)
#     df_b5 = poly_float(df_b4)
    df_a5 = df_a4
    df_b5 = df_b4
    
    def poly_dmy(df_0):
        df_1 = df_0.filter(like='dmy_')
        lst_col_0 = df_1.columns.to_list()
        poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
        arr_2 = poly.fit_transform(df_1)
        df_2 = pd.DataFrame(arr_2\
                            , index=df_1.index\
                            , columns=poly.get_feature_names_out())
        df_3 = df_2.drop(lst_col_0+['1'],axis='columns')
        col_0 = df_3.columns.to_list()
        col_1 = [re.sub(' ','_',s) for s in col_0]
        df_3.columns = col_1
        df_3 = df_3.add_prefix('poly_').astype(np.float)
        df_4 = pd.concat([df_0,df_3],axis='columns')
        return df_4
#     df_a6 = poly_float(df_a5)
#     df_b6 = poly_float(df_b5)
    df_a6 = df_a5
    df_b6 = df_b5
#     df_a6 = df_a5
#     df_b6 = df_a5
    
    def del_obj(df_0):
        lst_cols = df_0.select_dtypes(include='object')
        df_1 = df_0.drop(lst_cols, axis='columns')
        return df_1
    df_a7 = del_obj(df_a6)
    df_b7 = del_obj(df_b6)
    def func_(df_0):
        df_a = df_0.filter(like='dmy_')
        lst_cols = df_a.columns.to_list()
        poly = PolynomialFeatures(degree=3,interaction_only=True,include_bias=False)
        df_b = pd.DataFrame(poly.fit_transform(df_a)\
                     , index=df_a.index, columns=poly.get_feature_names_out())\
            .drop(lst_cols, axis='columns')
        lst_cols = df_b.columns.to_list()
        lst_cols = [re.sub(' ','_',s) for s in lst_cols]
        df_b.columns = lst_cols
        df_c = pd.concat([df_0,df_b],axis='columns')
        df_c
        return df_c
    df_a8 = func_(df_a7)
    df_b8 = func_(df_b7)
#     df_a8 = df_a7
#     df_b8 = df_a7
    
    def func_(df_0):
        df_1 = df_0[['Age_Years','Bilirubin']]
        df_a = pd.Series(index=df_1.index, name='flag_Bilirubin')
        df_a.loc[(df_1['Age_Years']>=18)&(df_1['Bilirubin']<1.2)] = 0
        df_a.loc[(df_1['Age_Years']<18)&(df_1['Bilirubin']<1.0)] = 0
        df_a = df_a.fillna(1.0)
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a9 = func_(df_a8)
    df_b9 = func_(df_b8)
    
    def func_(df_0):
        df_1 = df_0['Cholesterol']
        df_a = pd.Series(index=df_1.index, name='flag_Cholesterol')
        df_a.loc[(df_1<200)] = 0
        df_a.loc[(df_1>=200)&(df_1<240)] = 1
        df_a.loc[(df_1>=240)] = 2
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a10 = func_(df_a9)
    df_b10 = func_(df_b9)
    
    def func_(df_0):
        df_1 = df_0['Albumin']
        df_a = pd.Series(index=df_1.index, name='flag_Albumin')
        df_a.loc[(df_1<3.4)] = 1
        df_a.loc[(df_1>=3.4)&(df_1<=5.4)] = 0
        df_a.loc[(df_1>5.4)] = 2
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a11 = func_(df_a10)
    df_b11 = func_(df_b10)
    
    def func_(df_0):
        df_1 = df_0['Copper']
        df_a = pd.Series(index=df_1.index, name='flag_Copper')
        df_a.loc[(df_1<62)] = 1
        df_a.loc[(df_1>=62)&(df_1<=140)] = 0
        df_a.loc[(df_1>140)] = 2
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a12 = func_(df_a11)
    df_b12 = func_(df_b11)
    
    def func_(df_0):
        df_1 = df_0[['Age_Years','Alk_Phos']]
        df_a = pd.Series(index=df_1.index, name='flag_Alk_Phos')
        df_a.loc[(df_1['Age_Years']>=18)&(df_1['Alk_Phos']>1470)] = 2
        df_a.loc[(df_1['Age_Years']>=18)&(df_1['Alk_Phos']<44)] = 1
        df_a = df_a.fillna(0)
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a13 = func_(df_a12)
    df_b13 = func_(df_b12)
    
    def func_(df_0):
        df_1 = df_0[['cate_Sex','SGOT']]
        df_a = pd.Series(index=df_1.index, name='flag_SGOT')
        df_a.loc[(df_1['cate_Sex']=='M')&(df_1['SGOT']>50)] = 1
        df_a.loc[(df_1['cate_Sex']=='F')&(df_1['SGOT']>45)] = 1
        df_a = df_a.fillna(0)
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a14 = func_(df_a13)
    df_b14 = func_(df_b13)
    
    def func_(df_0):
        df_1 = df_0['Tryglicerides']
        df_a = pd.Series(index=df_1.index, name='flag_Tryglicerides')
        df_a.loc[(df_1<150)] = 0
        df_a.loc[(df_1>=150)&(df_1<200)] = 1
        df_a.loc[(df_1>=200)&(df_1<500)] = 2
        df_a.loc[(df_1>=500)] = 3
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a15 = func_(df_a14)
    df_b15 = func_(df_b14)
    
    def func_(df_0):
        df_1 = df_0['Platelets']
        df_a = pd.Series(index=df_1.index, name='flag_Platelets')
        df_a.loc[(df_1<150)] = 2
        df_a.loc[(df_1>450)] = 1
        df_a = df_a.fillna(0)
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a16 = func_(df_a15)
    df_b16 = func_(df_b15)
    
    def func_(df_0):
        df_1 = df_0['Prothrombin']
        df_a = pd.Series(index=df_1.index, name='flag_Prothrombin')
        df_a.loc[(df_1>=11)&(df_1<=13.5)] = 0
        df_a = df_a.fillna(1)
        df_b = pd.concat([df_0,df_a],axis='columns')
        return df_b
    df_a17 = func_(df_a16)
    df_b17 = func_(df_b16)
    return df_a17, df_b17

df_train_X, df_test_X = func_prepro_0(df_X_train, df_X_test)


In [14]:
enc = LabelEncoder()
arr_y = enc.fit_transform(df_y)
df_train_y = pd.Series(arr_y, index=df_y.index, name='Status')

In [15]:
# df_X_org = df_train_X.copy()
# df_y_org = df_train_y.copy()
# rus = RandomUnderSampler(random_state=17)
# df_train_X, df_train_y = rus.fit_resample(df_X_org, df_y_org)
# df_train_X.shape, df_X_org.shape,df_X_train.shape


In [16]:
df_train_y.value_counts()

Status
0    4965
2    2665
1     275
Name: count, dtype: int64

In [17]:
df_cv_X, df_e_X, df_cv_y, df_e_y = train_test_split(\
                                                   df_train_X, df_train_y\
                                                    , test_size=0.2\
                                                    , shuffle=True\
                                                    , stratify=df_train_y\
                                                    , random_state=17\
                                                   )
df_t_X = df_test_X.copy()
skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=17)

In [18]:
df_t_X.index

Index([ 7905,  7906,  7907,  7908,  7909,  7910,  7911,  7912,  7913,  7914,
       ...
       13166, 13167, 13168, 13169, 13170, 13171, 13172, 13173, 13174, 13175],
      dtype='int64', name='id', length=5271)

In [19]:
df_cv_X.shape, df_e_X.shape,df_t_X.shape

((6324, 416), (1581, 416), (5271, 416))

In [20]:
pd.Series(df_cv_X.columns.to_numpy()).value_counts()

cate_Drug                                                1
cate_Sex                                                 1
dmy_Sex_F_dmy_Edema_N_dmy_Edema_S                        1
dmy_Sex_F_dmy_Spiders_Y_dmy_Edema_Y                      1
dmy_Sex_F_dmy_Spiders_Y_dmy_Edema_S                      1
                                                        ..
dmy_Drug_D-penicillamine_dmy_Sex_F_dmy_Hepatomegaly_N    1
dmy_Drug_D-penicillamine_dmy_Sex_F_dmy_Ascites_Y         1
dmy_Drug_D-penicillamine_dmy_Sex_F_dmy_Ascites_N         1
dmy_Drug_D-penicillamine_dmy_Sex_F_dmy_Sex_M             1
flag_Prothrombin                                         1
Name: count, Length: 416, dtype: int64

In [21]:
df_cv_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6324 entries, 3325 to 2682
Columns: 416 entries, cate_Drug to flag_Prothrombin
dtypes: category(6), float64(393), int64(17)
memory usage: 19.9 MB


## Model

In [22]:
def feval_(preds,train):
    df_p0 = pd.DataFrame(np.array(preds).reshape(dct_cvet['df_v_y'].shape[0],-1))
    df_t0 = pd.Series(train.get_label())
    score = func_logloss(df_t0, df_p0)
    return 'score',score,False

In [23]:
class ModelLGBM2():
    def __init__(self):
        self.Model = lgb
    def preprocess(self, df_0):
        lst_col_cate = df_0.filter(like='cate_').columns.to_list()
        df_1 = df_0.drop(lst_col_cate, axis='columns')
        return df_1
    def train(self, df_X, df_y, skf):
        Model = self.Model
        df_cv_X = self.preprocess(df_X)
        df_cv_y = df_y
        df_c_X, df_v_X, df_c_y, df_v_y = train_test_split(\
                                                         df_cv_X, df_cv_y\
                                                          , test_size=0.25\
                                                          , shuffle=True\
                                                          , stratify=df_cv_y\
                                                          , random_state=17\
                                                         )
        params = {\
                  'objective':'multiclass'\
                  , 'num_class':3\
                  , 'verbosity':-1\
                  , 'metric':'multi_logloss'
#                   , 'metric':None\
                 }
        ds_c = Model.Dataset(df_c_X, df_c_y)
        ds_v = Model.Dataset(df_v_X, df_v_y)
        def objective(trial):
            max_depth_ = trial.suggest_int('max_depth', 3,7)
            num_leaves_ = trial.suggest_int('num_leaves',8,32)
            learning_rate_ = trial.suggest_float('learning_rate', 1e-2,1e-1,log=False)
            params_trial = {\
                      'max_depth':max_depth_\
                      , 'learning_rate':learning_rate_\
                     }
            params.update(params_trial)
            cb_log_evaluation = Model.log_evaluation(period=101)
            cb_early_stopping = Model.early_stopping(stopping_rounds=25,verbose=False)
            model = Model.train(\
                        params\
                        , ds_c\
                        , valid_sets=[ds_v]\
                        , callbacks=[cb_log_evaluation, cb_early_stopping]\
                        , num_boost_round=200\
#                         , feval=feval_\
                       )
#             arr_c = model.predict(df_c_X)
#             df_p_c = pd.DataFrame(arr_c, index=df_c_X.index)
#             score_c = func_logloss(df_c_y, df_p_c)          

            arr_v = model.predict(df_v_X)
            df_p_v = pd.DataFrame(arr_v, index=df_v_X.index)
            score_v = func_logloss(df_v_y, df_p_v)
            
#             score = ((score_v - score_c) - 0.001)**2
            return score_v
        sampler_ = optuna.samplers.TPESampler(seed=17)
        optuna.logging.set_verbosity(optuna.logging.CRITICAL)
        study = optuna.create_study(sampler=sampler_)
        study.optimize(objective, n_trials=100)
        self.study = study
        params.update(study.best_params)
        print(params)
        cb_log_evaluation = Model.log_evaluation(period=101)
        cb_early_stopping = Model.early_stopping(stopping_rounds=25,verbose=False)
        lst_models = []
        df_v_proba = pd.DataFrame()
        for idx_c, idx_v in skf.split(df_cv_X, df_cv_y):
            df_c_X = df_cv_X.iloc[idx_c]
            df_c_y = df_cv_y.iloc[idx_c]
            df_v_X = df_cv_X.iloc[idx_v]
            df_v_y = df_cv_X.iloc[idx_v]
            ds_c = Model.Dataset(df_c_X, df_c_y)
            ds_v = Model.Dataset(df_v_X, df_v_y)
            model = Model.train(\
                             params\
                              , ds_c\
#                               , valid_sets=[ds_v]\
                              , callbacks=[cb_log_evaluation]\
                              , num_boost_round=200\
                             )
            lst_models.append(model)
            arr_v = model.predict(df_v_X)
            df_0 = pd.DataFrame(arr_v, index=df_v_X.index)
            df_v_proba = pd.concat([df_v_proba, df_0],axis='index')
        self.df_v_proba = df_v_proba
        self.lst_models = lst_models
    def proba(self, df_0):
        df_X = self.preprocess(df_0)
        for i, model in enumerate(self.lst_models):
            arr_0 = model.predict(df_X)
            if i==0:
                arr_s = arr_0
            else:
                arr_s += arr_0
        arr_1 = arr_s/len(self.lst_models)
        df_1 = pd.DataFrame(arr_1, index=df_X.index)
        return df_1

modelLGBM2 = ModelLGBM2()
modelLGBM2.train(df_cv_X, df_cv_y, skf)

[101]	valid_0's multi_logloss: 0.479311
[101]	valid_0's multi_logloss: 0.461383
[101]	valid_0's multi_logloss: 0.498247
[101]	valid_0's multi_logloss: 0.506129
[101]	valid_0's multi_logloss: 0.490928
[101]	valid_0's multi_logloss: 0.455162
[101]	valid_0's multi_logloss: 0.461758
[101]	valid_0's multi_logloss: 0.456193
[101]	valid_0's multi_logloss: 0.475006
[101]	valid_0's multi_logloss: 0.476589
[101]	valid_0's multi_logloss: 0.453638
[101]	valid_0's multi_logloss: 0.45571
[101]	valid_0's multi_logloss: 0.45761
[101]	valid_0's multi_logloss: 0.454617
[101]	valid_0's multi_logloss: 0.457342
[101]	valid_0's multi_logloss: 0.453916
[101]	valid_0's multi_logloss: 0.466881
[101]	valid_0's multi_logloss: 0.456463
[101]	valid_0's multi_logloss: 0.459155
[101]	valid_0's multi_logloss: 0.466892
[101]	valid_0's multi_logloss: 0.457632
[101]	valid_0's multi_logloss: 0.457112
[101]	valid_0's multi_logloss: 0.457643
[101]	valid_0's multi_logloss: 0.46307
[101]	valid_0's multi_logloss: 0.454614
[10

In [24]:
df_p0 = modelLGBM2.proba(df_e_X)
df_t0 = df_e_y
print(func_logloss(df_t0, df_p0))
func_result(df_t0, df_p0)

0.43435613733692746
True     1313
False     268
Name: count, dtype: int64
0.83


In [25]:
df_0 = modelLGBM2.proba(df_t_X)
df_0.columns = ['Status_C','Status_CL','Status_D']
df_submit = df_0.reset_index(drop=False)
display(df_submit)
df_submit.to_csv('submission.csv',index=None)

,id,Status_C,Status_CL,Status_D
0,7905,0.598267,0.024522,0.377210
1,7906,0.553753,0.237389,0.208858
2,7907,0.009711,0.003774,0.986515
3,7908,0.937304,0.004415,0.058281
4,7909,0.778259,0.052389,0.169352
...,...,...,...,...
5266,13171,0.810478,0.154252,0.035270
5267,13172,0.972027,0.002369,0.025604
5268,13173,0.858242,0.009933,0.131825
5269,13174,0.993032,0.001501,0.005468


In [26]:
class ModelLGBM():
    def __init__(self):
        self.Model = lgb
    def preprocess(self, df_0):
        lst_col_dmy = df_0.filter(like='dmy_').columns.to_list()
        df_1 = df_0.drop(lst_col_dmy, axis='columns')
        return df_1
    def train(self, df_X, df_y, skf):
        Model = self.Model
        df_cv_X = self.preprocess(df_X)
        df_cv_y = df_y
        df_c_X, df_v_X, df_c_y, df_v_y = train_test_split(\
                                                         df_cv_X, df_cv_y\
                                                          , test_size=0.25\
                                                          , shuffle=True\
                                                          , stratify=df_cv_y\
                                                          , random_state=17\
                                                         )
        params = {\
                  'objective':'multiclass'\
                  , 'num_class':3\
                  , 'verbosity':-1\
                  , 'metric':'multi_logloss'
#                   , 'metric':None\
                 }
        ds_c = Model.Dataset(df_c_X, df_c_y)
        ds_v = Model.Dataset(df_v_X, df_v_y)
        def objective(trial):
            max_depth_ = trial.suggest_int('max_depth', 3,7)
            num_leaves_ = trial.suggest_int('num_leaves',8,32)
            learning_rate_ = trial.suggest_float('learning_rate', 1e-2,1e-1,log=False)
            params_trial = {\
                      'max_depth':max_depth_\
                      , 'learning_rate':learning_rate_\
                     }
            params.update(params_trial)
            cb_log_evaluation = Model.log_evaluation(period=101)
            cb_early_stopping = Model.early_stopping(stopping_rounds=25,verbose=False)
            model = Model.train(\
                        params\
                        , ds_c\
                        , valid_sets=[ds_v]\
                        , callbacks=[cb_log_evaluation, cb_early_stopping]\
#                         , feval=feval_\
                       )
#             arr_c = model.predict(df_c_X)
#             df_p_c = pd.DataFrame(arr_c, index=df_c_X.index)
#             score_c = func_logloss(df_c_y, df_p_c)          

            arr_v = model.predict(df_v_X)
            df_p_v = pd.DataFrame(arr_v, index=df_v_X.index)
            score_v = func_logloss(df_v_y, df_p_v)
            
#             score = ((score_v - score_c) - 0.001)**2
            return score_v
        sampler_ = optuna.samplers.TPESampler(seed=17)
        optuna.logging.set_verbosity(optuna.logging.CRITICAL)
        study = optuna.create_study(sampler=sampler_)
        study.optimize(objective, n_trials=100)
        self.study = study
        params.update(study.best_params)
        print(params)
        cb_log_evaluation = Model.log_evaluation(period=101)
        cb_early_stopping = Model.early_stopping(stopping_rounds=25,verbose=False)
        lst_models = []
        df_v_proba = pd.DataFrame()
        for idx_c, idx_v in skf.split(df_cv_X, df_cv_y):
            df_c_X = df_cv_X.iloc[idx_c]
            df_c_y = df_cv_y.iloc[idx_c]
            df_v_X = df_cv_X.iloc[idx_v]
            df_v_y = df_cv_X.iloc[idx_v]
            ds_c = Model.Dataset(df_c_X, df_c_y)
            model = Model.train(\
                             params\
                              , ds_c\
                              , callbacks=[cb_log_evaluation]\
                             )
            lst_models.append(model)
            arr_v = model.predict(df_v_X)
            df_0 = pd.DataFrame(arr_v, index=df_v_X.index)
            df_v_proba = pd.concat([df_v_proba, df_0],axis='index')
        self.df_v_proba = df_v_proba
        self.lst_models = lst_models
    def proba(self, df_0):
        df_X = self.preprocess(df_0)
        for i, model in enumerate(self.lst_models):
            arr_0 = model.predict(df_X)
            if i==0:
                arr_s = arr_0
            else:
                arr_s += arr_0
        arr_1 = arr_s/len(self.lst_models)
        df_1 = pd.DataFrame(arr_1, index=df_X.index)
        return df_1

# modelLGBM = ModelLGBM()
# modelLGBM.train(df_cv_X, df_cv_y, skf)

In [27]:
# df_p0 = modelLGBM.proba(df_e_X)
# df_t0 = df_e_y
# print(func_logloss(df_t0, df_p0))
# func_result(df_t0, df_p0)

In [28]:
class ModelSVC(ModelLGBM):
    def __init__(self):
        super().__init__()
        self.MODEL = SVC
    class objective:
        def __init__(self, MODEL, X, y):
            self.X = X
            self.y = y
            self.MODEL = MODEL
        def __call__(self,trial):
            MODEL = self.MODEL
            df_c_X,df_v_X,df_c_y,df_v_y=train_test_split(\
                                                        self.X,self.y\
                                                         , test_size=0.2\
                                                         , shuffle=True\
                                                         , stratify=self.y\
                                                         , random_state=17\
                                                        )
            C_ = trial.suggest_float('C',0.9,1.1)
            gamma_ = trial.suggest_categorical('gamma',['scale','auto'])
            tol_ = trial.suggest_float('tol',1e-4,1e-2, log=True)
            params = {\
                     'C':C_\
                      , 'gamma':gamma_\
                      , 'tol':tol_\
                     }
            model = self.MODEL(**params)
            OVR = OneVsRestClassifier(model)
            OVR.fit(df_c_X, df_c_y)
            arr_0 = OVR.decision_function(df_v_X)
            arr_1 = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_1, index=df_v_X.index)
            score = func_logloss(df_v_y, df_0)
            return score
    def preprocess(self, df_0):
        lst_col_cate = df_0.filter(like='cate_').columns.to_list()
        df_1 = df_0.drop(lst_col_cate, axis='columns')
        sclr = StandardScaler()
        arr_1 = sclr.fit_transform(df_1)
        df_2 = pd.DataFrame(arr_1, index=df_1.index, columns=df_1.columns)
        return df_2
    def train(self, df_X, df_y, skf):
        df_cv_X = self.preprocess(df_X)
        df_cv_y = df_y

        sampler_ = optuna.samplers.TPESampler(seed=17)
        optuna.logging.set_verbosity(optuna.logging.CRITICAL)
        study = optuna.create_study(sampler=sampler_, direction='minimize')
        study.optimize(self.objective(self.MODEL,df_cv_X,df_cv_y,), n_trials=100)
        params = study.best_params
        print(params)
        lst_models = []
        df_v_proba = pd.DataFrame()
        for idx_c, idx_v in skf.split(df_cv_X, df_cv_y):
            df_c_X = df_cv_X.iloc[idx_c]
            df_c_y = df_cv_y.iloc[idx_c]
            df_v_X = df_cv_X.iloc[idx_v]
            df_v_y = df_cv_y.iloc[idx_v]
            model = self.MODEL(**params)
            OVR = OneVsRestClassifier(model)
            OVR.fit(df_c_X, df_c_y)
            lst_models.append(OVR)
            arr_0 = OVR.decision_function(df_v_X)
            arr_v = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_v, index=df_v_X.index)
            df_v_proba = pd.concat([df_v_proba, df_0],axis='index')
        self.df_v_proba = df_v_proba
        self.lst_models = lst_models 
    def proba(self, df_0):
        df_X = self.preprocess(df_0)
        for i, model in enumerate(self.lst_models):
            arr_0 = model.decision_function(df_X)
            if i==0:
                arr_s = arr_0
            else:
                arr_s += arr_0
        arr_1 = arr_s/len(self.lst_models)
        arr_2 = softmax(arr_1, axis=1)
        df_1 = pd.DataFrame(arr_2, index=df_X.index)
        return df_1

# modelSVC = ModelSVC()
# modelSVC.train(df_cv_X, df_cv_y, skf)

In [29]:
# df_p0 = modelSVC.proba(df_e_X)
# df_t0 = df_e_y
# print(func_logloss(df_t0, df_p0))
# func_result(df_t0, df_p0)

In [30]:
class ModelSVC2(ModelLGBM):
    def __init__(self):
        super().__init__()
        self.MODEL = SVC
    class objective:
        def __init__(self, MODEL, X, y):
            self.X = X
            self.y = y
            self.MODEL = MODEL
        def __call__(self,trial):
            MODEL = self.MODEL
            df_c_X,df_v_X,df_c_y,df_v_y=train_test_split(\
                                                        self.X,self.y\
                                                         , test_size=0.2\
                                                         , shuffle=True\
                                                         , stratify=self.y\
                                                         , random_state=17\
                                                        )
            C_ = trial.suggest_float('C',0.9,1.1)
            gamma_ = trial.suggest_categorical('gamma',['scale','auto'])
            tol_ = trial.suggest_float('tol',1e-4,1e-2, log=True)
            params = {\
                     'C':C_\
                      , 'gamma':gamma_\
                      , 'tol':tol_\
                     }
            model = self.MODEL(**params)
            OVO = OneVsOneClassifier(model)
            OVO.fit(df_c_X, df_c_y)
            arr_0 = OVO.decision_function(df_v_X)
            arr_1 = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_1, index=df_v_X.index)
            score = func_logloss(df_v_y, df_0)
            return score
    def preprocess(self, df_0):
        lst_col_cate = df_0.filter(like='cate_').columns.to_list()
        df_1 = df_0.drop(lst_col_cate, axis='columns')
        sclr = StandardScaler()
        arr_1 = sclr.fit_transform(df_1)
        df_2 = pd.DataFrame(arr_1, index=df_1.index, columns=df_1.columns)
        return df_2
    def train(self, df_X, df_y, skf):
        df_cv_X = self.preprocess(df_X)
        df_cv_y = df_y

        sampler_ = optuna.samplers.TPESampler(seed=17)
        optuna.logging.set_verbosity(optuna.logging.CRITICAL)
        study = optuna.create_study(sampler=sampler_, direction='minimize')
        study.optimize(self.objective(self.MODEL,df_cv_X,df_cv_y,), n_trials=100)
        params = study.best_params
        print(params)
        lst_models = []
        df_v_proba = pd.DataFrame()
        for idx_c, idx_v in skf.split(df_cv_X, df_cv_y):
            df_c_X = df_cv_X.iloc[idx_c]
            df_c_y = df_cv_y.iloc[idx_c]
            df_v_X = df_cv_X.iloc[idx_v]
            df_v_y = df_cv_y.iloc[idx_v]
            model = self.MODEL(**params)
            OVO = OneVsRestClassifier(model)
            OVO.fit(df_c_X, df_c_y)
            lst_models.append(OVO)
            arr_0 = OVO.decision_function(df_v_X)
            arr_v = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_v, index=df_v_X.index)
            df_v_proba = pd.concat([df_v_proba, df_0],axis='index')
        self.df_v_proba = df_v_proba
        self.lst_models = lst_models 
    def proba(self, df_0):
        df_X = self.preprocess(df_0)
        for i, model in enumerate(self.lst_models):
            arr_0 = model.decision_function(df_X)
            if i==0:
                arr_s = arr_0
            else:
                arr_s += arr_0
        arr_1 = arr_s/len(self.lst_models)
        arr_2 = softmax(arr_1, axis=1)
        df_1 = pd.DataFrame(arr_2, index=df_X.index)
        return df_1

# modelSVC2 = ModelSVC2()
# modelSVC2.train(df_cv_X, df_cv_y, skf)

In [31]:
# df_p0 = modelSVC2.proba(df_e_X)
# df_t0 = df_e_y
# print(func_logloss(df_t0, df_p0))
# func_result(df_t0, df_p0)

In [32]:
class ModelSGD(ModelSVC):
    def __init__(self):
        super().__init__()
        self.MODEL = SGD
    class objective:
        def __init__(self, MODEL, X, y):
            self.X = X
            self.y = y
            self.MODEL = MODEL
        def __call__(self,trial):
            df_c_X,df_v_X,df_c_y,df_v_y=train_test_split(\
                                                        self.X,self.y\
                                                         , test_size=0.2\
                                                         , shuffle=True\
                                                         , stratify=self.y\
                                                         , random_state=17\
                                                        )
            loss_ = trial.suggest_categorical('loss',[\
                                                      'hinge'\
                                                      ,'log_loss'\
                                                      ,'modified_huber'\
                                                      ,'squared_hinge'\
                                                      ,'perceptron'\
                                                      ,'squared_error'\
                                                      ,'huber'\
                                                      ,'epsilon_insensitive'\
                                                      ,'squared_epsilon_insensitive'])
            penalty_ = trial.suggest_categorical('penalty',['l2','l1'])
            alpha_ = trial.suggest_float('alpha',1e-5,1e-3,log=True)
            params = {\
                     'loss':loss_\
                      , 'penalty':penalty_\
                      , 'alpha':alpha_\
                     }
            model = self.MODEL(**params)
            OVR = OneVsRestClassifier(model)
            OVR.fit(df_c_X, df_c_y)
            arr_0 = OVR.decision_function(df_v_X)
            arr_1 = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_1, index=df_v_X.index)
            score = func_logloss(df_v_y, df_0)
            return score

# modelSGD = ModelSGD()
# modelSGD.train(df_cv_X, df_cv_y, skf)

In [33]:
# df_p0 = modelSGD.proba(df_e_X)
# df_t0 = df_e_y
# print(func_logloss(df_t0, df_p0))
# func_result(df_t0, df_p0)

In [34]:
class ModelDTC(ModelSVC):
    def __init__(self):
        super().__init__()
        self.MODEL = DTC
    class objective:
        def __init__(self, MODEL, X, y):
            self.X = X
            self.y = y
            self.MODEL = MODEL
        def __call__(self,trial):
            df_c_X,df_v_X,df_c_y,df_v_y=train_test_split(\
                                                        self.X,self.y\
                                                         , test_size=0.2\
                                                         , shuffle=True\
                                                         , stratify=self.y\
                                                         , random_state=17\
                                                        )
            criterion_ = trial.suggest_categorical('criterion',[\
                                                      'gini'\
                                                      ,'entropy'\
                                                      ,'log_loss'\
                                                      ])
            max_depth_ = trial.suggest_int('max_depth',2,9)
            random_state_ = trial.suggest_int('random_state',0,11)
            params = {\
                     'criterion':criterion_\
                      , 'max_depth':max_depth_\
                      , 'random_state':random_state_\
                     }
            model = self.MODEL(**params)
            OVR = OneVsRestClassifier(model)
            OVR.fit(df_c_X, df_c_y)
            arr_0 = OVR.predict_proba(df_v_X)
            arr_1 = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_1, index=df_v_X.index)
            score = func_logloss(df_v_y, df_0)
            return score
    def train(self, df_X, df_y, skf):
        df_cv_X = self.preprocess(df_X)
        df_cv_y = df_y

        sampler_ = optuna.samplers.TPESampler(seed=17)
        optuna.logging.set_verbosity(optuna.logging.CRITICAL)
        study = optuna.create_study(sampler=sampler_, direction='minimize')
        study.optimize(self.objective(self.MODEL,df_cv_X,df_cv_y,), n_trials=100)
        params = study.best_params
        print(params)
        lst_models = []
        df_v_proba = pd.DataFrame()
        for idx_c, idx_v in skf.split(df_cv_X, df_cv_y):
            df_c_X = df_cv_X.iloc[idx_c]
            df_c_y = df_cv_y.iloc[idx_c]
            df_v_X = df_cv_X.iloc[idx_v]
            df_v_y = df_cv_y.iloc[idx_v]
            model = self.MODEL(**params)
            OVR = OneVsRestClassifier(model)
            OVR.fit(df_c_X, df_c_y)
            lst_models.append(OVR)
            arr_0 = OVR.predict_proba(df_v_X)
            arr_v = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_v, index=df_v_X.index)
            df_v_proba = pd.concat([df_v_proba, df_0],axis='index')
        self.df_v_proba = df_v_proba
        self.lst_models = lst_models 
    def proba(self, df_0):
        df_X = self.preprocess(df_0)
        for i, model in enumerate(self.lst_models):
            arr_0 = model.predict_proba(df_X)
            if i==0:
                arr_s = arr_0
            else:
                arr_s += arr_0
        arr_1 = arr_s/len(self.lst_models)
        arr_2 = softmax(arr_1, axis=1)
        df_1 = pd.DataFrame(arr_2, index=df_X.index)
        return df_1
# modelDTC = ModelDTC()
# modelDTC.train(df_cv_X, df_cv_y, skf)

In [35]:
# df_p0 = modelDTC.proba(df_e_X)
# df_t0 = df_e_y
# print(func_logloss(df_t0, df_p0))
# func_result(df_t0, df_p0)

In [36]:
def NN_MODEL(shape_,dense0_,dense1_,dense2_,dense3_,dense4_,dout0_,dout1_):
    model = keras.Sequential()
    model.add(layers.Input(shape=shape_))
    model.add(layers.Dense(dense0_, activation='relu'))
    model.add(layers.Dense(dense1_, activation='relu'))
    model.add(layers.Dropout(dout0_))
    model.add(layers.Dense(dense2_, activation='relu'))
    model.add(layers.Dense(dense3_, activation='relu'))
    model.add(layers.Dense(dense4_, activation='relu'))
    model.add(layers.Dropout(dout1_))
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(\
         optimizer='adam'\
          , loss=keras.losses.SparseCategoricalCrossentropy()\
          , metrics=['accuracy']
         )
    return model

In [37]:
class ModelTF():
    def __init__(self):
        pass
#         super().__init__()
#         self.MODEL = keras.Sequential()
    def preprocess(self,df_0):
        lst_col_cate = df_0.filter(like='cate_').columns.to_list()
        df_1 = df_0.drop(lst_col_cate, axis='columns')
        sclr = StandardScaler()
        arr_1 = sclr.fit_transform(df_1)
        df_2 = pd.DataFrame(arr_1, index=df_1.index, columns=df_1.columns)
        return df_2
    class objective:
        def __init__(self, X, y):
            self.X = X
            self.y = y
        def __call__(self,trial):
            df_c_X,df_v_X,df_c_y,df_v_y=train_test_split(\
                                                        self.X,self.y\
                                                         , test_size=0.2\
                                                         , shuffle=True\
                                                         , stratify=self.y\
                                                         , random_state=17\
                                                        )
            tf_c_X = tf.constant(df_c_X)
            tf_c_y = tf.constant(df_c_y.to_frame())
            tf_v_X = tf.constant(df_v_X)
            tf_v_y = tf.constant(df_v_y.to_frame())
            
            shape_ = df_c_X.shape[1]
            dense0_ = trial.suggest_int('dense0_',3,256)
            dense1_ = trial.suggest_int('dense1_',3,256)
            dense2_ = trial.suggest_int('dense2_',3,256)
            dense3_ = trial.suggest_int('dense3_',3,256)
            dense4_ = trial.suggest_int('dense4_',3,256)
            dout0_ = trial.suggest_float('dout0_',0.1,0.5)
            dout1_ = trial.suggest_float('dout1_',0.1,0.5)
            
            params = {\
                     'shape_':shape_\
                     , 'dense0_':dense0_\
                     , 'dense1_':dense1_\
                     , 'dense2_':dense2_\
                     , 'dense3_':dense3_\
                     , 'dense4_':dense4_\
                     , 'dout0_':dout0_\
                     , 'dout1_':dout1_\
                     }
            model = NN_MODEL(**params)
            model.fit(tf_c_X, tf_c_y)
            arr_0 = model.predict(tf_v_X)
            arr_1 = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_1, index=df_v_X.index)
            score = func_logloss(df_v_y, df_0)
            return score
    def train(self, df_X, df_y, skf):
        df_cv_X = self.preprocess(df_X)
        df_cv_y = df_y

        sampler_ = optuna.samplers.TPESampler(seed=17)
        optuna.logging.set_verbosity(optuna.logging.CRITICAL)
        study = optuna.create_study(sampler=sampler_, direction='minimize')
        study.optimize(self.objective(df_cv_X,df_cv_y), n_trials=100)
        params = {'shape_':df_cv_X.shape[1]}
        params.update(study.best_params)
        print(params)
        model = NN_MODEL(**params)
        lst_models = []
        df_v_proba = pd.DataFrame()
        for idx_c, idx_v in skf.split(df_cv_X, df_cv_y):
            df_c_X = df_cv_X.iloc[idx_c]
            df_c_y = df_cv_y.iloc[idx_c]
            df_v_X = df_cv_X.iloc[idx_v]
            df_v_y = df_cv_y.iloc[idx_v]

            model.fit(df_c_X, df_c_y)
            lst_models.append(model)
            arr_0 = model.predict(df_v_X)
            arr_v = softmax(arr_0, axis=1)
            df_0 = pd.DataFrame(arr_v, index=df_v_X.index)
            df_v_proba = pd.concat([df_v_proba, df_0],axis='index')
        self.df_v_proba = df_v_proba
        self.lst_models = lst_models 
    def proba(self, df_0):
        df_X = self.preprocess(df_0)
        for i, model in enumerate(self.lst_models):
            arr_0 = model.predict(df_X)
            if i==0:
                arr_s = arr_0
            else:
                arr_s += arr_0
        arr_1 = arr_s/len(self.lst_models)
        arr_2 = softmax(arr_1, axis=1)
        df_1 = pd.DataFrame(arr_2, index=df_X.index)
        return df_1

# modelTF = ModelTF()
# modelTF.train(df_cv_X, df_cv_y, skf)

In [38]:
# df_p0 = modelTF.proba(df_e_X)
# df_t0 = df_e_y
# print(func_logloss(df_t0, df_p0))
# func_result(df_t0, df_p0)

In [39]:
lst_model = [modelLGBM2]
lst_str_model = ['LGBM2']
# lst_model = [modelLGBM, modelLGBM2, modelSVC, modelSVC2, modelSGD, modelDTC]
# lst_str_model = ['LGBM','LGBM2','SVC','SVC2','SGD','DTC']
for model in lst_model:
    df_p0 = model.proba(df_e_X)
    df_t0 = df_e_y
    score = func_logloss(df_t0, df_p0)
    print(score)
    func_result(df_t0, df_p0)

0.43435613733692746
True     1313
False     268
Name: count, dtype: int64
0.83


In [40]:
df_t_X.index

Index([ 7905,  7906,  7907,  7908,  7909,  7910,  7911,  7912,  7913,  7914,
       ...
       13166, 13167, 13168, 13169, 13170, 13171, 13172, 13173, 13174, 13175],
      dtype='int64', name='id', length=5271)

In [41]:
def make_X_2nd(flag='v'):
    df_X_2nd =pd.DataFrame()
    for i, model in enumerate(lst_model):
        if flag=='v':
            df_0 = model.df_v_proba
            df_0.index = model.df_v_proba.index
        elif flag=='e':
            df_0 = model.proba(df_e_X)
            df_0.index = df_e_X.index
        else:
            df_0 = model.proba(df_t_X)
            df_0.index = df_t_X.index
        df_1 = df_0.add_prefix(lst_str_model[i])
        df_X_2nd = pd.concat([df_X_2nd, df_1], axis='columns')
    return df_X_2nd
df_Xv = make_X_2nd('v').sort_index()
df_Xe = make_X_2nd('e').sort_index()
df_Xt = make_X_2nd('t').sort_index()

In [42]:
df_Xt

,LGBM20,LGBM21,LGBM22
id,,,
7905,0.598267,0.024522,0.377210
7906,0.553753,0.237389,0.208858
7907,0.009711,0.003774,0.986515
7908,0.937304,0.004415,0.058281
7909,0.778259,0.052389,0.169352
...,...,...,...
13171,0.810478,0.154252,0.035270
13172,0.972027,0.002369,0.025604
13173,0.858242,0.009933,0.131825


In [43]:
df_yv = df_cv_y.sort_index()
df_ye = df_e_y.sort_index()

In [44]:
dfcvX,dfeX,dfcvy,dfey = train_test_split(df_Xv,df_yv\
                                        , test_size=0.2\
                                        , shuffle=True\
                                        , stratify=df_yv\
                                        , random_state=17)
skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=17)

In [45]:
modelLGBM2nd = ModelLGBM()
modelLGBM2nd.train(dfcvX,dfcvy, skf)

{'objective': 'multiclass', 'num_class': 3, 'verbosity': -1, 'metric': 'multi_logloss', 'max_depth': 3, 'learning_rate': 0.08585796138534992, 'num_leaves': 20}


In [46]:
df_p0 = modelLGBM2nd.proba(df_Xe)
df_t0 = df_ye
func_logloss(df_t0,df_p0)

0.4515460570594575

In [47]:
func_result(df_t0, df_p0)

True     1311
False     270
Name: count, dtype: int64
0.829


In [48]:
# df_0 = modelLGBM2nd.proba(df_Xt)
# df_0.columns = ['Status_C','Status_CL','Status_D']
# df_submit = df_0.reset_index(drop=False)
# df_submit
# df_submit.to_csv('submission.csv',index=None)